# T5 tuning & inference



The T5 model was proposed in the work: ["Exploring the Limits of Transfer Learning with a Unified
Text-to-Text Transformer"](https://arxiv.org/pdf/1910.10683.pdf). The idea behind this model is to explore the insights regarding transfer learning and creating a model which would be able to solve different tasks (like paraphrasing, text classification etc.) out of box. The resulting encoder-decoder model showed a state-of-art performance on various datasets.

This notebooks finetunes and evaluates the performance for the T5 encoder-decoder. The two approaches to fine-tuning were taken:
1. Standard tuning with labeled data
2. Reinforcement learning-based finetuning

In [1]:
!pip install -q datasets transformers[sentencepiece] sacrebleu
!pip install -q evaluate
!pip install -q langchain
!pip install -q sentence-transformers
!pip install -q git+http://github.com/huggingface/trl.git@b6b593d61af124bfda111c1cb1370d9166236ea5
# !pip install -q -U trl

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Necessary inputs
import warnings

import gc
from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from tqdm import tqdm
from tqdm.auto import trange
tqdm.pandas()

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, \
                         DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, \
                         RobertaTokenizer, RobertaForSequenceClassification
import wandb
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead, set_seed

import evaluate
warnings.filterwarnings('ignore')
transformers.logging.set_verbosity_warning()
transformers.logging.set_verbosity_error()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
# 0fb8270bb27d51de8c3cedee367d3455496c0021
wandb.init()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Part 1. Labelled tuning

## Loading the dataset

In [7]:
transformers.set_seed(42)
dataset = datasets.load_dataset("domrachev03/toxic_comments_subset")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/domrachev03--toxic_comments_subset-480257536bff0e3f/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
n_train = 10000
n_val = 1000
ds_subset = dataset['train'].select(range(n_train + n_val))

dataset['train'] = ds_subset.select(range(n_train)) 
dataset['val'] = ds_subset.select(range(n_train, n_train+n_val))

In [9]:
max_input_length = 64
max_target_length = 64
batch_size = 128

In [10]:
# simple postprocessing for textgithub
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


In [40]:
import gc
import tqdm
from tqdm.auto import trange
import torch
import numpy as np

from transformers import AutoModelForSequenceClassification, AutoTokenizer, \
    RobertaTokenizer, RobertaForSequenceClassification

import evaluate


def cleanup():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def get_toxicity(preds, soft=False, batch_size=1, device='cuda'):
    results = []

    model_name = 'SkolkovoInstitute/roberta_toxicity_classifier'

    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaForSequenceClassification.from_pretrained(model_name)
    device = device
    model.to(device)

    model.eval()
    for i in tqdm.tqdm(range(0, len(preds), batch_size)):
        batch = tokenizer(preds[i:i + batch_size], return_tensors='pt', max_length=-1, padding=True).to(device)

        with torch.no_grad():
            logits = model(**batch).logits
            out = torch.softmax(logits, -1)[:, 1].cpu().numpy()
            results.append(out)
    return 1 - np.concatenate(results)


def get_sacrebleu(inputs, preds):
    metric = evaluate.load("sacrebleu")

    result = metric.compute(predictions=preds, references=inputs)
    return result['score']


def get_fluency(preds, soft=False, batch_size=1, device='cuda'):
    path = 'cointegrated/roberta-large-cola-krishna2020'

    model = RobertaForSequenceClassification.from_pretrained(path)
    tokenizer = AutoTokenizer.from_pretrained(path)
    device = device
    model.to(device)

    results = []
    for i in trange(0, len(preds), batch_size):
        batch = [t for t in preds[i: i + batch_size]]
        inputs = tokenizer(batch, max_length=-1, padding=True, return_tensors='pt').to(device)
        with torch.no_grad():
            out = torch.softmax(model(**inputs).logits, -1)[:, 0].cpu().numpy()
            results.append(out)
    return np.concatenate(results)


def compute_metrics(eval_preds, tokenizer=None, print_results=False, batch_size=1, device='cuda'):
    preds, labels = eval_preds
    
    if tokenizer is not None:
        detokenized_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        filtered_labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        detokenized_labels = tokenizer.batch_decode(filtered_labels, skip_special_tokens=True)
    else:
        detokenized_preds = preds
        detokenized_labels = labels

    results = {}
    results['toxic'] = get_toxicity(detokenized_preds, batch_size=batch_size, device=device)
    results['avg_toxic'] = sum(results['toxic']) / len(results['toxic'])
    cleanup()

    results['bleu'] = get_sacrebleu(detokenized_labels, detokenized_preds) / 100
    cleanup()

    results['fluency'] = get_fluency(detokenized_preds, batch_size=batch_size, device=device)
    results['avg_fluency'] = sum(results['fluency']) / len(results['fluency'])
    cleanup()

    # count metrics
    results['joint'] = sum(results['toxic'] * results['bleu'] * results['fluency']) / len(preds)
    if print_results:
        print("--------------")
        print("Metric   | Value")
        print("--------------")
        print(f"toxic    | {results['avg_toxic']:.2f}")
        print(f"bleu (n) | {results['bleu']:.2f}")
        print(f"fluency  | {results['avg_fluency']:.2f}")
        print("===============")
        print(f"Total    | {results['joint']:.2f}")
        print("--------------")
    return results

100%|██████████| 3/3 [00:00<00:00, 84.16it/s]


  0%|          | 0/3 [00:00<?, ?it/s]

--------------
Metric   | Value
--------------
toxic    | 0.33
bleu (n) | 0.09
fluency  | 0.89
Total    | 0.03
--------------


# T5-small

## Selecting the model
The baseline model is [`t5-small`](https://huggingface.co/docs/transformers/model_doc/t5) transformer. This acritecture seems suitable for our task, since: "*T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, e.g., for translation: translate English to German: ..., for summarization: summarize: ...*"

In [12]:
# selecting model checkpoint
model_type = "t5-small"

The model utilizes pretrained tokenizer

In [13]:
t5_tokenizer = AutoTokenizer.from_pretrained(model_type, )

def t5_wrapper(text):
    wrapped = f"Make the following sentence non-toxic: '{text}'"
    return wrapped

def t5_preprocess_function(examples):
    inputs = t5_wrapper(examples["reference"])
    targets = examples["translation"]
    
    model_inputs = t5_tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = t5_tokenizer(targets, max_length=max_target_length, truncation=True)

    return model_inputs


In [14]:
t5_ds_train = dataset['train'].map(
    t5_preprocess_function, 
    batched=False,
    load_from_cache_file=False,
)
t5_ds_train.set_format(type="torch")

  0%|          | 0/10000 [00:00<?, ?ex/s]

## T5 reinforcement learning

The `huggingface` provides a software for performing a [Transformer Reinforcement Learning (TRL)](https://huggingface.co/docs/trl/index). However, it seems like this approach is highly uninvestigated (I've managed to find a few systematic reviews on the area). Hence, I decided to give it a try on a given task

## The metric

In [15]:
from sentence_transformers import util

def get_toxicity_reward(preds, batch_size=1):
    results = []

    model_name = 'SkolkovoInstitute/roberta_toxicity_classifier'

    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaForSequenceClassification.from_pretrained(model_name)
    device = 'cuda'
    model.to(device)

    model.eval()
    for i in tqdm(range(0, len(preds), batch_size)):
        batch = tokenizer(preds[i:i + batch_size], return_tensors='pt', max_length=-1, padding=True).to(device)

        with torch.no_grad():
            logits = model(**batch).logits
            out = logits[:, 0]
            results.append(out)
    return torch.concatenate(results)

# def get_cosine_similarity_reward(inputs, preds):
#     return np.array([util.pytorch_cos_sim(in_i.type(torch.float64), pred_i.type(torch.float64)) for in_i, pred_i in zip(inputs, preds)])

def get_sacrebleu(inputs, preds):
    metric = evaluate.load("sacrebleu")

    results = []
    for i in range(len(inputs)):
        results.append(metric.compute(predictions=preds[i:i+1], references=inputs[i:i+1])['score'])
    return torch.tensor(results).to('cuda')


def get_fluency_reward(preds, batch_size=1):
    path = 'cointegrated/roberta-large-cola-krishna2020'

    model = RobertaForSequenceClassification.from_pretrained(path)
    tokenizer = AutoTokenizer.from_pretrained(path)
    device = 'cuda'
    model.to(device)

    results = []
    for i in trange(0, len(preds), batch_size):
        batch = [t for t in preds[i: i + batch_size]]
        inputs = tokenizer(batch, max_length=-1, padding=True, return_tensors='pt').to(device)
        with torch.no_grad():
            out = model(**inputs).logits[:, 0]
            results.append(out)
    return torch.concatenate(results)


def compute_reward(preds, labels, batch_size=1):
    toxicity = get_toxicity_reward(preds, batch_size=batch_size)
    cleanup()

    bleu = torch.where(get_sacrebleu(labels, preds) < 0.2, -20, 0)
    cleanup()

    fluency = get_fluency_reward(preds, batch_size=batch_size)
    cleanup()
    return toxicity + bleu + fluency


In [16]:
compute_reward(
    ['fuck you, bitch, I love you!', '<extra_id_0>. Retest:</s>'], 
    ['fuck you, bitch, I hate you!', 'I Love you so much I could not express it!'], 
    batch_size=2
)

100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([ -2.0337, -14.2195], device='cuda:0')

## The train process

In [17]:
config = PPOConfig(
    model_name="t5-small",
    learning_rate=1.41e-5,
    ppo_epochs=4,
    batch_size=128,
    log_with='wandb',
)

set_seed(42)


In [18]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [19]:
t5_ds_train

Dataset({
    features: ['reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [20]:
cols_to_remove = ['reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox']

t5_clean_ds = t5_ds_train.remove_columns(cols_to_remove)

t5_clean_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [21]:
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)


In [22]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=t5_clean_ds, data_collator=collator)

In [23]:
generation_kwargs = {
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "min_new_tokens": 4
}

stat_logs = []
batch_log = []
rewards_log = []
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), total=len(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    #### Get response from t5
    response_tensors = []
    for query in query_tensors:
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze())
    batch["response"] = [tokenizer.decode(r[1:].squeeze()) for r in response_tensors]
    
    # response = ppo_trainer.generate(query_tensors, **generation_kwargs)  
    # batch["response"] = [tokenizer.decode(r[1:]) for r in response]
    batch['query'] = [tokenizer.decode(inp) for inp in batch['input_ids']]
    batch['query'] = [q[q.find('\'')+1: q.rfind('\'')] for q in batch['query']]
    rewards = compute_reward(batch['response'], batch['query'], batch_size=batch_size)
    rewards = [torch.tensor([reward]) for reward in rewards]

    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

100%|██████████| 1/1 [00:00<00:00, 13.93it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 14.10it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 15.07it/s]


...

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 50.56it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

 99%|█████████▊| 78/79 [1:16:53<00:59, 59.15s/it]


ValueError: Batch size (128) does not match number of examples - but got 16 for: queries

In [34]:
model.save_pretrained("t5_rl_best", push_to_hub=False)

## Evaluation

In [38]:
model.from_pretrained('/kaggle/working/t5_rl_best')
model.eval()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

results = []
msgs = [t5_wrapper(text) for text in dataset["test"]["reference"]]
for i in trange(0, len(msgs), batch_size):
    batch = msgs[i: i+batch_size]
    tokenized_batch = tokenizer(batch, max_length=128, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        output = model.generate(tokenized_batch['input_ids'])
        result = [tokenizer.decode(out_i, skip_special_tokens=True,temperature=0) for out_i in output]
    results.extend(result)

  0%|          | 0/136 [00:00<?, ?it/s]

In [41]:
compute_metrics((results, dataset['test']['translation']), print_results=True, batch_size=batch_size)

100%|██████████| 136/136 [00:14<00:00,  9.27it/s]


  0%|          | 0/136 [00:00<?, ?it/s]

--------------
Metric   | Value
--------------
toxic    | 0.93
bleu (n) | 0.03
fluency  | 0.49
Total    | 0.01
--------------


{'toxic': array([0.99982816, 0.99994403, 0.9999064 , ..., 0.99994403, 0.99994403,
        0.99982816], dtype=float32),
 'avg_toxic': 0.9251211043690382,
 'bleu': 0.02733905992001097,
 'fluency': array([0.06736284, 0.9749654 , 0.06025813, ..., 0.9749654 , 0.9749654 ,
        0.06736284], dtype=float32),
 'avg_fluency': 0.48509223179666894,
 'joint': 0.011723955617528401}

## Saving

In [ ]:
model.save_pretrained('domrachev03/t5_rl_detox')

In [42]:
import pandas as pd

preds_dict = pd.DataFrame([[orig_i, pred_i, label_i] for orig_i, pred_i, label_i in zip(dataset['test']['reference'], results, dataset['test']['translation'])], columns=['Query', 'Predictions', 'Labels'])

preds_dict.head(10)

,Query,Predictions,Labels
0,"If they say they don't want your help, you can...",the following sentence non-toxic.,"if they say they don't need help, you can bet ..."
1,"When you go home, they'il call you ""Mr. No-Leg...",non-toxic.,"when you get home, they'll call you Mr."
2,Mrs. Sanders ain't goingfor that late shit no ...,the sentence non-toxic.,Mrs. Sanders is not curious about being late.
3,I want every bloody one of them to hear.,the sentence non-toxic.,I want everyone to hear it.
4,That shit hurt.,.,and that really hurts.
5,How do I know if I tell you what you need to k...,non-toxic.,how do I know if I tell you what you want to k...
6,"This guy ain't going nowhere. Oh, shit!",the following sentence non-toxic!,this guy's not going anywhere.
7,Fucked if I can remember.,.,if only I could remember.
8,I fucked one stitch and he looks at me now as ...,'I made the following sentence non-toxic: I ma...,"I mess up a suture, and now he's looking at me..."
9,Earth! Meet my lovely assistant... Tiny tits.,the following sentence non-toxic.,"please welcome my beautiful assistant, Maloprs..."


In [43]:
preds_dict.to_csv('t5_rl_test.csv')